In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

pd.options.mode.chained_assignment = None  # default='warn' exclude warning for chained assingments

In [4]:
pd.set_option('display.float_format', lambda x: '%.4f' % x) ## change float at the macro level

In [5]:
def BlackScholes (rf, spot, strike, T, sigma, div_yield):
    
    """""""""
    rf - risk free rate;
    spot - current spot price of the target asset;
    strike - option strike price;
    T - time, measured in years. For instance 1 month would be 1/12;
    sigma - expected annual volatility. For instance, 100% annual volatility would mean sigma in our model equals 1.
    
    c = S0*exp(-kT)*N(d1) - K*exp(-rfT)*N(d2)
    p = K*exp(-rfT)*N(-d2) - S0*exp(-kT)*N(-d1)

    """""""""
    
    d1 = (np.log(spot/strike) + T * (rf + (sigma**2)/2)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    neg_d1 = -d1
    neg_d2 = -d2
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    N_neg_d1 = norm.cdf(neg_d1)
    N_neg_d2 = norm.cdf(neg_d2)
    
    exp1 = np.exp(-rf*T)
    exp2 = np.exp(-div_yield*T)
    
    
    call_price = spot  * exp2 * N_d1 - strike * exp1 * N_d2
    put_price = strike * exp1 * N_neg_d2 - spot * exp2 * N_neg_d1
    
    data = pd.DataFrame(data = [call_price, put_price], columns = ['Price'], index = ['Call', 'Put'])

    return data

In [6]:
def European_Binomial (rf, spot, strike, T, sigma, N, div_yield):
    
    """""""""
    rf - risk free rate;
    spot - current spot price of the target asset;
    strike - option strike price;
    T - time, measured in years. For instance 1 month would be 1/12;
    sigma - expected annual volatility. For instance, 100% annual volatility would mean sigma in our model equals 1.
    N - number of binomial steps;
    
    The factor by which the price raises: u = exp (std * sqrt(delta t));
    The factor by which the price falls: d = 1/u;
    Probability of price going up: pu = [exp(delta t * (rf - k)) - d] / [u - d];
    Probability of pirce going down: pd = 1 - pu;

    Stock price at the end of the period according to binomial model is:
    S0 * u ^ (# of steps up) * d ^ (# of steps down)
    """""""""
    
    delta_T = T/N
    
    NR = np.exp((rf - div_yield) * delta_T)
    GR = np.exp(rf*delta_T)
    
    u = np.exp(sigma * np.sqrt(delta_T))
    d = 1/u

    p_up = (NR - d) / (u - d) 
    p_down = 1 - p_up

    discount = np.exp(-rf*T)    
    
    df = pd.DataFrame(data = np.arange(0, N+1), columns = ["Number of down steps"])
    df['S(t)'] = spot * u ** (N - df['Number of down steps']) * d ** (df["Number of down steps"])

    call_price = []
    put_price = []
    for i in range(len(df)):
        call_price.append(max(0, float(df['S(t)'].iloc[i:i+1] - strike)))
        put_price.append(max(0, float(strike - df['S(t)'].iloc[i:i+1])))

    df["Call (t)"] = call_price
    df['Put (t)'] =  put_price

    df["Probability for the nod"] = p_up ** (N - df["Number of down steps"]) * p_down ** (df["Number of down steps"])

    df["Number"] = N

    df["Number Chosen"] = df["Number"] - df["Number of down steps"]

    paths = []
    for i in range(len(df)):
        paths.append ((np.math.factorial(float(df["Number"].iloc[i:i+1]))) / (np.math.factorial(float(df["Number Chosen"].iloc[i:i+1])) * np.math.factorial(float(df["Number"].iloc[i:i+1]) - float(df["Number Chosen"].iloc[i:i+1]))))

    df["Number of paths"] = paths

    df['Pr of S{t}'] = df["Probability for the nod"] * df["Number of paths"]

    df["Call (t) * Pr"] = df["Call (t)"] * df["Pr of S{t}"]
    df["Put (t) * Pr"] = df["Put (t)"] * df["Pr of S{t}"]
    
    data = pd.DataFrame(data = [df["Call (t) * Pr"].sum() * discount, df["Put (t) * Pr"].sum() * discount], columns = ['Price'], index = ['Call', 'Put'])
    
    return data, df

In [7]:
rf, spot, strike, T, sigma, N, div_yield = 0.06, 10, 8, 15/252, 0.4, 10, 0.02

input_frame = pd.DataFrame(data = [rf, spot, strike, T, sigma, N, div_yield], columns=['Input'], 
                   index=['Risk-free rate', 'Spot', 'Strike', 'T', 'Volatility', 'Number of steps', 'Dividend Yield'])
display(input_frame)

,Input
Risk-free rate,0.0600
Spot,10.0000
Strike,8.0000
T,0.0595
Volatility,0.4000
Number of steps,10.0000
Dividend Yield,0.0200


In [8]:
European_Binomial(rf, spot, strike, T, sigma, N, div_yield)[0]

,Price
Call,2.0193
Put,0.0026


In [9]:
BlackScholes (rf, spot, strike, T, sigma, div_yield)

,Price
Call,2.0197
Put,0.0031


In [10]:
def American_Option (rf, spot, strike, T, sigma, N, div_yield):
    
    """""""""
    rf - risk free rate;
    spot - current spot price of the target asset;
    strike - option strike price;
    T - time, measured in years. For instance 1 month would be 1/12;
    sigma - expected annual volatility. For instance, 100% annual volatility would mean sigma in our model equals 1.
    N - number of binomial steps;
    
    The factor by which the price raises: u = exp (std * sqrt(delta t));
    The factor by which the price falls: d = 1/u;
    Probability of price going up: pu = [exp(delta t * (rf - k)) - d] / [u - d];
    Probability of pirce going down: pd = 1 - pu;

    Stock price at the end of the period according to binomial model is:
    S0 * u ^ (# of steps up) * d ^ (# of steps down)
    """""""""
    
    delta_T = T/N

    NR = np.exp((rf - div_yield) * delta_T)
    GR = np.exp(rf*delta_T)

    u = np.exp(sigma * np.sqrt(delta_T))
    d = 1/u

    p_up = (NR - d) / (u - d) 
    p_down = 1 - p_up  


    tree = pd.DataFrame(np.zeros((N+1, N+1)))
    tree.iloc[0, 0] = spot

    for i in range (1, N+1):
        tree.iloc[0:1][i] = tree.iloc[0:1][i-1] * u #step up

    for z in range(1, N+1):

        tree.iloc[z:z+1][z] = float(tree.iloc[z-1:z][z-1] * d)

        for i in range(z+1, N+1):
            tree.iloc[z:z+1][i] = tree.iloc[z:z+1][i-1] * u

    call_payoff = tree.copy()
    call_payoff.iloc[:, N][call_payoff.iloc[:, N] - strike < 0] = 0
    call_payoff.iloc[:, N][call_payoff.iloc[:, N] - strike > 0] -= strike

    put_payoff = tree.copy()
    put_payoff.iloc[:, N][strike - put_payoff.iloc[:, N] > 0] = strike - put_payoff.iloc[:, N][strike - put_payoff.iloc[:, N] > 0]
    put_payoff.iloc[:, N][strike - put_payoff.iloc[:, N] < 0] = 0

    for z in range(N):
        for i in range(N-z):
            num = N - z - 1
            call_payoff.iloc[:, num][i] = max(call_payoff.iloc[:, num][i] - strike, ((p_up * call_payoff.iloc[:, num+1][i] + p_down * call_payoff.iloc[:, num+1][i+1]) / np.exp(rf * delta_T)))
            put_payoff.iloc[:, num][i] = max(strike - put_payoff.iloc[:, num][i], ((p_up * put_payoff.iloc[:, num+1][i] + p_down * put_payoff.iloc[:, num+1][i+1]) / np.exp(rf * delta_T)))

    data = pd.DataFrame(data = [call_payoff.iloc[0, 0], put_payoff.iloc[0, 0]], columns = ['Price'], index = ['Call', 'Put'])
    
    return data

In [11]:
American_Option(rf, spot, strike, T, sigma, N, div_yield)

,Price
Call,2.0193
Put,0.0026


In [12]:
def Power_call(rf, spot, strike, T, sigma, N, div_yield, power):

    delta_T = T/N

    NR = np.exp((rf - div_yield) * delta_T)
    GR = np.exp(rf*delta_T)

    u = np.exp(sigma * np.sqrt(delta_T))
    d = 1/u

    p_up = (NR - d) / (u - d) 
    p_down = 1 - p_up  


    tree = pd.DataFrame(np.zeros((N+1, N+1)))
    tree.iloc[0, 0] = spot

    for i in range (1, N+1):
        tree.iloc[0:1][i] = tree.iloc[0:1][i-1] * u #step up

    for z in range(1, N+1):

        tree.iloc[z:z+1][z] = float(tree.iloc[z-1:z][z-1] * d)

        for i in range(z+1, N+1):
            tree.iloc[z:z+1][i] = tree.iloc[z:z+1][i-1] * u
            
    call_payoff = tree.copy()
    call_payoff.iloc[:, N][call_payoff.iloc[:, N] - strike < 0] = 0
    call_payoff.iloc[:, N][call_payoff.iloc[:, N] - strike > 0] -= strike

    call_payoff.iloc[:, N] = call_payoff.iloc[:, N] ** power
    
    for z in range(N):
        for i in range(N-z):
            num = N - z - 1
            call_payoff.iloc[:, num][i] = max(call_payoff.iloc[:, num][i] - strike, ((p_up * call_payoff.iloc[:, num+1][i] + p_down * call_payoff.iloc[:, num+1][i+1]) / np.exp(rf * delta_T)))
        
    return call_payoff.iloc[0, 0]

In [13]:
Power_call(rf, spot, strike, T, sigma, N, div_yield, power = 3)

14.345918104531002